
## The *args and **kwargs are the ones you passed from tuner.search().


In [ ]:



class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [16, 32]),
            **kwargs,
        )

tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="tune_hypermodel",
)



## For epochs specifically, I'd alternatively recommend looking at using early stopping during training via passing in the tf.keras.callbacks.EarlyStopping callback if it's applicable to your use case. This can be configured to stop your training as soon as the validation loss stops improving. You can pass Keras callbacks like this to search:

# Will stop training if the "val_loss" hasn't improved in 3 epochs.
tuner.search(x, y, epochs=30, callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)])

## Ref: https://github.com/keras-team/keras-tuner/issues/122


## Adding dropout layer to a model architecture - know how!



Keras automatically takes care of applying dropout during training and adjusting the layer's output at test time. When you add a tf.keras.layers.Dropout layer to your model, dropout is only active during training. This means that during training, a fraction of the features (as specified by the dropout rate) will be zeroed out at random to prevent overfitting by adding regularization to the network.

During testing, validation, or making predictions with model.predict() on new data, Keras automatically disables dropout. Therefore, all neurons will be active, and their outputs will not be zeroed out. However, as mentioned in the context you provided, the layer's output values are scaled down by a factor equal to the dropout rate at training time to compensate for the fact that more units are active than at training time. Keras handles this scaling automatically, so you don't need to adjust the model or specify anything additional when evaluating or using your model for predictions.

In summary, when using dropout in Keras:

During training: Dropout is applied, and a fraction of the inputs are randomly set to 0.
During testing, validation, or prediction: Dropout is not applied, but the outputs are properly scaled to account for the difference in active units between training and test time.
You do not need to worry about manually managing dropout when switching between training and testing phases; Keras manages this for you seamlessly.



## A LSTM Model
Ref: https://www.kaggle.com/code/anseldsouza/nvidia-price-prediction-lstm-using-keras-tuner


In [ ]:

def model_builder(hp):
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit',min_value=32,max_value=512,step=32), return_sequences=True, 
                   input_shape= ( X_train.shape[1],X_train.shape[2])))
    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
    model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32)))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(30, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
   
    model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])
    
    return model
    
tuner = kt.RandomSearch(model_builder, objective="mse", max_trials = 4, executions_per_trial =2,directory = "./")

tuner.search(x=X_train, y=y_train, epochs = 10, batch_size =256, validation_data=(X_test, y_test))


## In this example, they have used the best model from the tuning process to predict on the test data 

best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()
y_pred = best_model.predict(X_test)



## Another LSTM Model sample
Ref: https://kamran-afzali.github.io/posts/2022-02-20/tuner.html


In [ ]:

def build_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit',min_value=32,max_value=128,step=32),return_sequences=True, input_shape=(1,24)))
    for i in range(hp.Int('n_layers', 1, 10)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=128,step=32),return_sequences=True))
    model.add(LSTM(6))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(6))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])
    return model


In [ ]:

tuner= kt.RandomSearch(
        build_model,
        objective='mse',
        max_trials=10,
        executions_per_trial=3
        )


In [ ]:

tuner.search(
        x=X_train_t,
        y=c,
        epochs=20,
        batch_size=128,
        validation_data=(X_test_t,d),
)


In [ ]:

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


## Logs and checkpoints

The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.

